In [1]:
import os
from copy import deepcopy

from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from Environment import Environment, make_one_hot, give_mapping
from Networks import UserActor, AsstActor, CentralizedCritic

In [2]:
def give_prev_steps(prev_steps_assist, steps):
    prev_steps_assist = [[0,0,0,0,-1,-1] for i in range(steps-1)]
    return prev_steps_assist

In [3]:
class Agent:
    def __init__(self):
        self.memory_len = 6
        self.user_actor = UserActor()
        self.asst_actor = AsstActor(self.memory_len)
        self.user_critic = CentralizedCritic(self.memory_len)
        self.asst_critic = CentralizedCritic(self.memory_len)
        self.optimizer_actors = tf.keras.optimizers.Adam(lr = 0.0001)
        self.optimizer_critic = tf.keras.optimizers.Adam(lr = 0.0002)
        self.huber_loss = tf.keras.losses.Huber()
        
        
        self.gamma = 0.90
#         self.env = Environment()
#         self.env.cells = np.array([[0.1, 0.1], [0.9, 0.9], [0.1, 0.9], [0.9, 0.1], [0.5, 0.3], [0.5, 0.7]])
#         self.env_cell_mapping = give_mapping(self.env.cells)
#         self.env_cell_mapping = self.env_cell_mapping[np.newaxis, :, :, np.newaxis]
        self.eps = 10e-6
    
    def learn(self):
        max_steps = 40
        running_reward = 0
        reached = 0
        best = 0
        
        for epoch in tqdm(range(100000)):
            
            user_action_probs_history = []
            asst_action_probs_history = []
            user_critic_value_history = []
            asst_critic_value_history = []
            rewards_history = []
            returns = [] #Returns
            
            if epoch%100==0:
                self.env = Environment()
                env = self.env
        #         self.env.cells = np.array([[0.1, 0.1], [0.9, 0.9], [0.1, 0.9], [0.9, 0.1], [0.5, 0.3], [0.5, 0.7]])
                self.env_cell_mapping = give_mapping(self.env.cells)
                self.env_cell_mapping = self.env_cell_mapping[np.newaxis, :, :, np.newaxis]
            
            done = False
            episode_reward = 0
            start, dest = env.give_start_dest()
            ob_user = [start[0], start[1], dest[0], dest[1]]
            prev_steps_assist = []
            prev_steps_assist = give_prev_steps(prev_steps_assist, self.memory_len)
            step = 0
            episode_reward = 0
            
            with tf.GradientTape(persistent = True) as tape:
                while not done and step<max_steps:
                    curr_loc = ob_user[:2]
                    target_loc = ob_user[2:4]
                    step+=1
                    ob_user = np.array(ob_user)[np.newaxis]
                    user_probs = self.user_actor.model(ob_user)
                    user_action = np.random.choice(4, p=np.squeeze(user_probs))
                    user_action_probs_history.append(tf.math.log(user_probs[0, user_action]))

                    action_user_one_hot = make_one_hot(user_action, 4)

                    ob_assist = [action_user_one_hot + curr_loc] 
                    ob_assist = prev_steps_assist + ob_assist
                    ob_assist = np.array(ob_assist)[np.newaxis]
                    
                    asst_probs = self.asst_actor.model([ob_assist, self.env_cell_mapping])
                    asst_action = np.random.choice(4, p=np.squeeze(asst_probs))
                    asst_action_probs_history.append(tf.math.log(asst_probs[0, asst_action]))
                    
                    asst_output_one_hot = np.array(make_one_hot(asst_action, 4))[np.newaxis]
                    
                    user_critic_value = self.user_critic.model([ob_user, ob_assist, self.env_cell_mapping, asst_output_one_hot])
                    user_critic_value_history.append(user_critic_value)
                    
                    asst_critic_value = self.asst_critic.model([ob_user, ob_assist, self.env_cell_mapping, asst_output_one_hot])
                    asst_critic_value_history.append(asst_critic_value)
                    
                    new_loc, reward_user, reward_assist, done = self.env.step(user_action, asst_action + 1, target_loc, curr_loc)
                    
                    next_ob_user = new_loc[:]
                    next_ob_user = next_ob_user + target_loc

                    ob_user = next_ob_user
                    prev_steps_assist = np.squeeze(ob_assist).tolist()[1:]
                    
                    rewards_history.append(reward_user)
                    episode_reward+=reward_user
                    
                    if done:
                        reached += 1

                running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

                discounted_sum = 0
                for r in rewards_history[::-1]:
                    discounted_sum = r + self.gamma * discounted_sum
                    returns.append(discounted_sum)
                returns.reverse()
                
                user_critic_losses = []
                asst_critic_losses = []
                user_losses = []
                asst_losses = []
                
                for log_prob_user, log_prob_asst, val_user, val_asst, ret in zip(user_action_probs_history, asst_action_probs_history, user_critic_value_history,\
                                                                 asst_critic_value_history, returns):
                    diff_user = ret - val_user
                    diff_asst = ret - val_asst
                    user_losses.append(-log_prob_user*diff_user)
                    asst_losses.append(-log_prob_asst*diff_asst)
                    user_critic_losses.append(self.huber_loss(tf.expand_dims(val_user, 0), tf.expand_dims(ret, 0)))
                    asst_critic_losses.append(self.huber_loss(tf.expand_dims(val_asst, 0), tf.expand_dims(ret, 0)))

                user_loss = sum(user_losses)
                asst_loss = sum(asst_losses)
                user_critic_loss = sum(user_critic_losses)
                asst_critic_loss = sum(asst_critic_losses)
                
            grads = tape.gradient(user_loss, self.user_actor.model.trainable_variables)
            self.optimizer_actors.apply_gradients(zip(grads, self.user_actor.model.trainable_variables))

            grads = tape.gradient(asst_loss, self.asst_actor.model.trainable_variables)
            self.optimizer_actors.apply_gradients(zip(grads, self.asst_actor.model.trainable_variables))

            grads = tape.gradient(user_critic_loss, self.user_critic.model.trainable_variables)
            self.optimizer_critic.apply_gradients(zip(grads, self.user_critic.model.trainable_variables))
            
            grads = tape.gradient(asst_critic_loss, self.asst_critic.model.trainable_variables)
            self.optimizer_critic.apply_gradients(zip(grads, self.asst_critic.model.trainable_variables))
            
            
            if epoch%100 == 0:
                print(running_reward)
            
            if epoch and epoch%100 == 0:
                print(reached)
                if reached>=best:
                    best = reached
                    tf.keras.models.save_model(self.user_actor.model, 'user.h5')
                    tf.keras.models.save_model(self.asst_actor.model, 'asst.h5')
                    tf.keras.models.save_model(self.user_critic.model, 'user_critic.h5')
                    tf.keras.models.save_model(self.asst_critic.model, 'asst_critic.h5')
                    
                reached = 0
                
                if epoch%200 == 0:
                    self.trial()
                    
                    if epoch%1000 == 0:
                        if epoch<=10000:
                            max_steps -= 2
                            max_steps = max(max_steps, 10)
                        
                        elif epoch%2000 == 0:
                            max_steps -= 1
                            max_steps = max(max_steps, 6)
                            
                        
    def trial(self):
        env = self.env
        max_steps = 10
        done = False
        episode_reward = 0
        start, dest = env.give_start_dest()
        ob_user = [start[0], start[1], dest[0], dest[1]]
        prev_steps_assist = []
        prev_steps_assist = give_prev_steps(prev_steps_assist, self.memory_len)
        step = 0
        episode_reward = 0

        while not done and step<max_steps:
            curr_loc = ob_user[:2]
            target_loc = ob_user[2:4]
            step+=1
            print(ob_user)

            ob_user = np.array(ob_user)[np.newaxis]
            user_probs = self.user_actor.model(ob_user)
            user_action = np.argmax(np.squeeze(user_probs))

            action_user_one_hot = make_one_hot(user_action, 4)

            ob_assist = [action_user_one_hot + curr_loc] 
            ob_assist = prev_steps_assist + ob_assist
            ob_assist = np.array(ob_assist)[np.newaxis]

            asst_probs = self.asst_actor.model([ob_assist, self.env_cell_mapping])
            print(asst_probs)
            asst_action = np.argmax(np.squeeze(asst_probs)) 
            new_loc, reward_user, reward_assist, done = self.env.step(user_action, asst_action + 1, target_loc, curr_loc)

            next_ob_user = new_loc[:]
            next_ob_user = next_ob_user + target_loc

            ob_user = next_ob_user
            prev_steps_assist = np.squeeze(ob_assist).tolist()[1:]
            episode_reward+=reward_user




In [4]:
agent = Agent()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
subtract (Subtract)             (None, 2)            0           tf_op_layer_strided_slice[0][0]  
                                                                 tf_op_layer_strided_sl

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_4 (Te [(None, 2)]          0           input_8[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_5 (Te [(None, 2)]          0           input_8[0][0]                    
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 11, 11, 1)]  0                                            
_______________________________________________________________________________________

In [5]:
# agent.user_actor.model = tf.keras.models.load_model('user.h5')
# agent.user_critic.model = tf.keras.models.load_model('user_critic.h5')
# agent.asst_actor.model = tf.keras.models.load_model('asst.h5')
# agent.asst_critic.model = tf.keras.models.load_model('asst_critic.h5')

In [6]:
agent.learn()

  0%|                                                                                       | 0/100000 [00:00<?, ?it/s]

Icon Locations:
[[0.7 0.8]
 [0.  0.9]
 [0.2 0.1]
 [0.5 0.6]
 [0.7 0.6]
 [0.2 0.4]]
Icon usage Probabilities
[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]


  0%|                                                                           | 1/100000 [00:03<102:05:37,  3.68s/it]

0.25


  0%|                                                                          | 100/100000 [03:36<62:48:39,  2.26s/it]

...

100%|███████████████████████████████████████████████████████████████████████▉| 99900/100000 [13:46:46<00:29,  3.35it/s]

Icon Locations:
[[0.8 0.5]
 [0.2 0.1]
 [0.4 0.9]
 [0.3 0.6]
 [0.3 0.2]
 [0.2 0. ]]
Icon usage Probabilities
[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]


100%|███████████████████████████████████████████████████████████████████████▉| 99901/100000 [13:46:47<00:33,  2.95it/s]

6.205352264591247
100


100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [13:47:16<00:00,  2.01it/s]


In [7]:
agent.trial()

[0.7, 0.1, 0.3, 0.6]
tf.Tensor([[1.3289037e-09 1.7894278e-07 1.8863133e-04 9.9981111e-01]], shape=(1, 4), dtype=float32)
[0.7, 0.5, 0.3, 0.6]
tf.Tensor([[1.2123001e-09 2.0659066e-07 2.8567144e-04 9.9971408e-01]], shape=(1, 4), dtype=float32)
[0.3, 0.5, 0.3, 0.6]
tf.Tensor([[8.1702538e-09 4.6637515e-06 5.0914497e-03 9.9490392e-01]], shape=(1, 4), dtype=float32)
[0.3, 0.9, 0.3, 0.6]
tf.Tensor([[3.6581981e-04 9.8968279e-01 9.9446243e-03 6.6951766e-06]], shape=(1, 4), dtype=float32)
[0.3, 0.7, 0.3, 0.6]
tf.Tensor([[9.9980539e-01 1.8817424e-04 6.4284150e-06 5.0383488e-08]], shape=(1, 4), dtype=float32)
